# Validate PR #334 Support offset pitch for Safe mode and NSM events

In [1]:
import os
import sys
from pathlib import Path

# Need dev version of ska_sun
sys.path.insert(0, str(Path.home() / "git" / "kadi"))
sys.path.insert(0, str(Path.home() / "git" / "ska_sun"))

from cxotime import CxoTime

import kadi.commands as kc
import kadi.commands.states as kcs
from kadi.scripts import update_cmds_v2

In [2]:
kadi_dir = Path("nsm-ssm-offset")
kadi_dir.mkdir(exist_ok=True)
scenario = "test-scenario"

In [3]:
stop = CxoTime("2023:203")  # Clean period with no command events within +/- 50 days
os.environ["KADI"] = str(kadi_dir)  # Data root for reading/writing cmds2.h5,pkl
kc.conf.commands_dir = str(kadi_dir)  # Directory for cmd_events.csv and loads
os.environ["KADI_COMMANDS_DEFAULT_STOP"] = stop.date
os.environ["KADI_SCENARIO"] = scenario

In [4]:
cmd_events_path = kadi_dir / scenario / "cmd_events.csv"
print(cmd_events_path.read_text())

State,Date,Event,Params,Author,Reviewer,Comment
Definitive,2023:199:00:00:00.000,Safe mode,160,,,
Definitive,2023:200:00:00:00.000,NSM,,,,
Definitive,2023:200:12:00:00.000,Maneuver,0.80889172 -0.48342105 0.28726927 0.17168183,,,pitch=120 yaw=0 time=2023:200:12:00:00
Definitive,2023:201:00:00:00.000,Safe mode,,,,
Definitive,2023:201:12:00:00.000,NSM,140,,,



In [5]:
update_cmds_v2.main(
    (
        "--lookback=30",
        f"--stop={stop.date}",
        f"--data-root={kadi_dir}",
        f"--scenario={os.environ['KADI_SCENARIO']}",
    )
)

******************************************
Running: /Users/aldcroft/git/kadi/kadi/scripts/update_cmds_v2.py
Version: 7.12.1.dev7+g101ab24.d20241016
Time: Wed Oct 16 15:38:05 2024
User: root
Machine: saos-MacBook-Pro.local
Processing args:
{'data_root': 'nsm-ssm-offset',
 'log_level': 10,
 'lookback': 30,
 'scenario': 'test-scenario',
 'stop': '2023:203:00:00:00.000'}
******************************************
2024-10-16 15:38:05,496 get_cmd_events: Reading command events /Users/aldcroft/git/kadi/validate/pr334/nsm-ssm-offset/test-scenario/cmd_events.csv
2024-10-16 15:38:05,499 filter_cmd_events_default_stop: Filtering cmd_events to stop date 2023:203:00:00:00.000 (5 vs 5)
2024-10-16 15:38:05,502 get_occweb_page: Getting OCCweb FOT/mission_planning/PRODUCTS/APPR_LOADS/2023/JUN with cache=False
2024-10-16 15:38:05,648 get_occweb_page: Getting OCCweb FOT/mission_planning/PRODUCTS/APPR_LOADS/2023/JUN/JUN2623A with cache=False
2024-10-16 15:38:05,747 get_occweb_page: Getting OCCweb FOT/miss

In [6]:
cmds = kc.get_cmds("2023:199")
cmds = cmds[cmds["type"] != "ORBPOINT"]
cmds.pprint_like_backstop()

2023:199:00:00:00.000 | COMMAND_SW       | ACPCSFSU   | CMD_EVT  | event=Safe_mode, event_date=2023:199:00:00:00, pitch=160, scs=0
2023:199:00:00:00.000 | COMMAND_SW       | CODISASX   | CMD_EVT  | codisas1=128 , event=Safe_mode, event_date=2023:199:00:00:00, msid=CODISASX, scs
2023:199:00:00:00.000 | COMMAND_SW       | CODISASX   | CMD_EVT  | codisas1=129 , event=Safe_mode, event_date=2023:199:00:00:00, msid=CODISASX, scs
2023:199:00:00:00.000 | COMMAND_SW       | CODISASX   | CMD_EVT  | codisas1=130 , event=Safe_mode, event_date=2023:199:00:00:00, msid=CODISASX, scs
2023:199:00:00:00.000 | COMMAND_SW       | CODISASX   | CMD_EVT  | codisas1=131 , event=Safe_mode, event_date=2023:199:00:00:00, msid=CODISASX, scs
2023:199:00:00:00.000 | COMMAND_SW       | CODISASX   | CMD_EVT  | codisas1=132 , event=Safe_mode, event_date=2023:199:00:00:00, msid=CODISASX, scs
2023:199:00:00:00.000 | COMMAND_SW       | CODISASX   | CMD_EVT  | codisas1=133 , event=Safe_mode, event_date=2023:199:00:00:00, 

In [7]:
states = kcs.get_states(
    "2023:198:23:00:00", "2023:202:00:00:00", state_keys=["pitch", "pcad_mode"]
)
states["pitch"].info.format = ".1f"
states["datestart", "pitch", "pcad_mode", "trans_keys"].pprint_all()

      datestart       pitch pcad_mode    trans_keys  
--------------------- ----- --------- ---------------
2023:198:23:00:00.000 144.6      NPNT                
2023:199:00:00:00.000 146.2      SSUN pcad_mode,pitch
2023:199:00:04:14.544 152.3      SSUN           pitch
2023:199:00:08:29.087 158.4      SSUN           pitch
2023:199:00:12:43.631 160.0      SSUN           pitch
2023:200:00:00:00.000 158.1      NSUN pcad_mode,pitch
2023:200:00:05:20.553 146.4      NSUN           pitch
2023:200:00:10:41.105 125.4      NSUN           pitch
2023:200:00:16:01.658 104.3      NSUN           pitch
2023:200:00:21:22.210  92.7      NSUN           pitch
2023:200:00:26:42.763  90.0      NSUN           pitch
2023:200:12:00:00.000  90.0      NMAN       pcad_mode
2023:200:12:00:10.250  90.7      NMAN           pitch
2023:200:12:04:59.280  92.4      NMAN           pitch
2023:200:12:09:48.309  96.4      NMAN           pitch
2023:200:12:14:37.339 102.0      NMAN           pitch
2023:200:12:19:26.368 108.2 